In [5]:
# ⬇️ Google Drive & deps
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# === Clean & Pin: Colab 기본 스택과 호환되는 조합으로 맞춥니다. ===
import sys, subprocess, os

def pip(*args):
    print(">>", " ".join(args))
    subprocess.check_call([sys.executable, "-m", "pip", *args])

# 1) 충돌나는 핵심 패키지 제거
pip("uninstall", "-y", "pyarrow", "pandas", "numpy", "fastparquet", "scikit-learn")

# 2) 호환 조합으로 설치
# - Colab 기본: pandas==2.2.2 (google-colab가 이 버전을 고정 사용)
# - NumPy 2.0.2 유지 (Colab 기본)
# - pyarrow 16.x는 NumPy 2 대응 안정, pandas 2.2.2와도 호환 OK
# - scikit-learn 1.5.x로 상향(에러 메시지에서 요구)
pip("install", "-q", "numpy==2.0.2", "pandas==2.2.2", "pyarrow==16.1.0", "scikit-learn==1.5.2")

# 3) NLP 스택(Transformers/Accelerate)은 '의존성 자동 업그레이드'를 막습니다.
#    (google-colab 고정 패키지를 건드리지 않도록 --no-deps)
pip("install", "-q", "--no-deps", "transformers>=4.44.0", "accelerate>=0.33.0", "tqdm>=4.66.4", "pandas==2.2.2")

print("\n✅ Reinstall done. Now restarting kernel to finalize...")
os.kill(os.getpid(), 9)  # 🔁 런타임 강제 재시작


>> uninstall -y pyarrow pandas numpy fastparquet scikit-learn
>> install -q numpy==2.0.2 pandas==2.2.2 pyarrow==16.1.0 scikit-learn==1.5.2
>> install -q --no-deps transformers>=4.44.0 accelerate>=0.33.0 tqdm>=4.66.4 pandas==2.2.2


In [3]:
# -*- coding: utf-8 -*-
"""KcELECTRA_MTL_A100_Turbo_v3.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1XZRGRb7w8_OaWlzltY8NaJEnnXmXwK_f

# 🚀 KcELECTRA MTL — A100 Turbo (v3, FP32-cast fix)

**Shared encoder + 2 heads** (Urgency 3-class, Sentiment 4-class)

- ✅ Call-level F1 (chunk→call 로짓 평균)
- ✅ AMP 최신 문법 (`torch.amp.autocast('cuda', ...)`)
- ✅ BF16/FP16 사용 시 **넘파이 변환 전 FP32 캐스팅**(이번 버그 픽스)
- ✅ MAX_LEN/STRIDE 스윕 도우미 (튜닝판)
- ✅ DataLoader 튜닝 / 로컬 캐시 (터보판)

- ✅ /content 로컬 캐시, Drive엔 체크포인트만
- ✅ BF16 + TF32, 큰 배치, 콜레벨 평가만 매 epoch
"""

# -*- coding: utf-8 -*-
"""KcELECTRA_MTL_Improved.ipynb

🚀 개선된 KcELECTRA MTL - A100 최적화 버전

주요 개선사항:
- ✅ 클래스 불균형 강화 (Sentiment 0: 2.3% → 가중치 증가)
- ✅ 메모리 효율성 개선
- ✅ Early Stopping + Best Model 복원
- ✅ 클래스별 F1 상세 출력
- ✅ 캐시 에러 처리 강화
- ✅ 평가 속도 개선
"""

import torch, platform, warnings
warnings.filterwarnings('ignore')
print("PyTorch:", torch.__version__, "| CUDA:", torch.version.cuda)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
print("Python:", platform.python_version())

# ⚙️ Config
import os, random, numpy as np, pandas as pd
from pathlib import Path

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# 경로 설정
DATA_DIR  = Path("/content/drive/MyDrive/project1/Processed_Data/MTL_ready_v3")  # ✅ v3 사용
SAVE_DIR  = Path("/content/drive/MyDrive/project1/kcelectra_mtl_ckpt_v3")
CACHE_DIR = Path("/content/kcelectra_mtl_cache")
for d in [SAVE_DIR, CACHE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# AMP 설정
bf16_ok = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
amp_dtype = torch.bfloat16 if bf16_ok else torch.float16
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    try:
        torch.set_float32_matmul_precision("medium")
    except:
        pass

# 하이퍼파라미터
BATCH_SIZE = 160
GRAD_ACCUM = 1
NUM_WORKERS = 4  # MODIFIED: 8→4로 줄여 메모리 안정성 높임 (A100에서 데드 방지)
PREFETCH = 4
PIN_MEMORY = True
PERSISTENT = True

MAX_LEN = 384
STRIDE = 192

EPOCHS = 15
LR = 3e-5
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01
MAX_GRAD_NORM = 1.0
LABEL_SMOOTHING = 0.1
DROPOUT_RATE = 0.3

# ✅ Task 가중치 조정 (Sentiment가 더 어려움)
ALPHA = 0.4  # Urgency (균형잡힘)
BETA = 0.6   # Sentiment (불균형)

# ✅ 클래스 가중치 gamma (MODIFIED: Urgency 낮춤 (균형적), Sentiment 증가)
GAMMA_URGENCY = 0.3  # 균형 데이터라 낮춤
GAMMA_SENT = 0.8     # 클래스 0 강화

MODEL_NAME = "beomi/KcELECTRA-base-v2022"
print(f"\n📋 Config:")
print(f"  AMP dtype: {amp_dtype}")
print(f"  MAX_LEN/STRIDE: {MAX_LEN}/{STRIDE}")
print(f"  BATCH: {BATCH_SIZE} | GRAD_ACCUM: {GRAD_ACCUM}")
print(f"  Workers: {NUM_WORKERS} | Prefetch: {PREFETCH}")
print(f"  Task weights: α={ALPHA}, β={BETA}")
print(f"  Class gamma: Urgency={GAMMA_URGENCY}, Sentiment={GAMMA_SENT}")

# 📥 Load MTL v3 data
def read_any(base: Path, stem: str):
    """Parquet → CSV → Feather 순으로 시도"""
    for ext, fn in [(".parquet", pd.read_parquet),
                    (".csv", pd.read_csv),
                    (".feather", pd.read_feather)]:
        fp = base / f"{stem}{ext}"
        if fp.exists():
            print(f"  → {fp.name}")
            try:
                return fn(fp)
            except Exception as e:
                print(f"⚠️ Failed to read {fp.name}: {e}. Try installing engines or checking versions.")
    raise FileNotFoundError(f"{stem} not found in {base}")

print("\n📂 Loading data...")
X_train = read_any(DATA_DIR, "X_train")
X_test  = read_any(DATA_DIR, "X_test")
y_u_tr  = read_any(DATA_DIR, "y_train_urgency")
y_u_te  = read_any(DATA_DIR, "y_test_urgency")
y_s_tr  = read_any(DATA_DIR, "y_train_sentiment")
y_s_te  = read_any(DATA_DIR, "y_test_sentiment")

def squeeze_label(obj, key=None):
    if isinstance(obj, pd.DataFrame):
        if key and key in obj.columns:
            return obj[key].astype(int)
        if obj.shape[1] == 1:
            return obj.iloc[:, 0].astype(int)
    return pd.Series(obj).astype(int)

y_u_tr = squeeze_label(y_u_tr, "y_urgency")
y_u_te = squeeze_label(y_u_te, "y_urgency")
y_s_tr = squeeze_label(y_s_tr, "y_sentiment")
y_s_te = squeeze_label(y_s_te, "y_sentiment")

text_tr = X_train["full_text"].astype(str)
text_te = X_test["full_text"].astype(str)

print(f"✅ Data loaded: Train={len(text_tr):,}, Test={len(text_te):,}")

# ✅ 클래스 분포 확인
print("\n📊 Class Distribution:")
print("Urgency (Train):", y_u_tr.value_counts(normalize=True).sort_index().to_dict())
print("Sentiment (Train):", y_s_tr.value_counts(normalize=True).sort_index().to_dict())

# 🔤 Tokenizer & Model
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn

print(f"\n🔤 Loading tokenizer: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

class ElectraMTL(nn.Module):
    def __init__(self, base_name: str, n_u: int = 3, n_s: int = 4, dropout=0.3):
        super().__init__()
        self.base = AutoModel.from_pretrained(base_name)
        hdim = self.base.config.hidden_size
        self.dropout = nn.Dropout(dropout)
        self.head_u = nn.Linear(hdim, n_u)
        self.head_s = nn.Linear(hdim, n_s)

    def forward(self, input_ids, attention_mask):
        out = self.base(input_ids=input_ids, attention_mask=attention_mask)
        cls = out.last_hidden_state[:, 0]
        z = self.dropout(cls)
        return self.head_u(z), self.head_s(z)

print("🏗️ Building model...")
model = ElectraMTL(MODEL_NAME, n_u=3, n_s=4, dropout=DROPOUT_RATE).to(device)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"  Total params: {total_params:,}")
print(f"  Trainable params: {trainable_params:,}")

# 🧱 Tokenize + Chunk Cache
from tqdm.auto import tqdm

def build_chunk_cache_MTL(texts, y_u, y_s, tokenizer, split_name: str,
                          max_len=512, stride=256, batch_size=1024):
    """청크 단위로 토크나이징 + 캐싱"""
    cache_path = CACHE_DIR / f"{split_name}_ml{max_len}_st{stride}.pt"

    # ✅ 캐시 존재 시 로딩 (MODIFIED: dtype/shape 무결성 체크 추가)
    if cache_path.exists():
        try:
            print(f"⚡ Loading cached chunks: {cache_path.name}")
            cached = torch.load(cache_path, weights_only=False)
            expected_keys = {"input_ids", "attention_mask", "y_u", "y_s", "sample_idx"}
            if expected_keys.issubset(cached.keys()):
                # 추가 체크: dtype과 shape
                if cached["input_ids"].dtype == torch.int32 and cached["attention_mask"].dtype == torch.int8:
                    print(f"  Chunks: {cached['y_u'].size(0):,}")
                    return cached
                else:
                    print("  ⚠️ Cache dtype/shape invalid, regenerating...")
            else:
                print("  ⚠️ Invalid cache, regenerating...")
        except Exception as e:
            print(f"  ⚠️ Cache load failed: {e}, regenerating...")

    # 캐시 생성
    print(f"🔨 Building cache: {split_name} (MAX_LEN={max_len}, STRIDE={stride})")
    all_ids, all_masks, all_u, all_s, all_map = [], [], [], [], []
    N = len(texts)

    for i in tqdm(range(0, N, batch_size), desc=f"Tokenize [{split_name}]"):
        batch_texts = [str(t) for t in texts[i:i+batch_size]]

        enc = tokenizer(
            batch_texts,
            max_length=max_len,
            truncation=True,
            padding="max_length",
            return_attention_mask=True,
            return_overflowing_tokens=True,
            stride=stride,
            return_tensors="pt"
        )

        mapping = enc.pop("overflow_to_sample_mapping")
        global_map = (mapping + i).to(torch.int32)

        # 메모리 효율적인 타입 사용
        ids = enc["input_ids"].to(torch.int32)
        mask = enc["attention_mask"].to(torch.int8)

        labs_u = torch.tensor([int(y_u.iloc[int(j)]) for j in global_map],
                              dtype=torch.int64)
        labs_s = torch.tensor([int(y_s.iloc[int(j)]) for j in global_map],
                              dtype=torch.int64)

        all_ids.append(ids)
        all_masks.append(mask)
        all_u.append(labs_u)
        all_s.append(labs_s)
        all_map.append(global_map)

    tensors = {
        "input_ids": torch.cat(all_ids, 0),
        "attention_mask": torch.cat(all_masks, 0),
        "y_u": torch.cat(all_u, 0),
        "y_s": torch.cat(all_s, 0),
        "sample_idx": torch.cat(all_map, 0),
    }

    torch.save(tensors, cache_path)
    print(f"💾 Saved cache → {cache_path.name} | Chunks: {tensors['y_u'].size(0):,}")
    return tensors

# 캐시 생성/로딩
print("\n🔄 Preparing chunks...")
tr_cache = build_chunk_cache_MTL(text_tr, y_u_tr, y_s_tr, tokenizer, "train", MAX_LEN, STRIDE)
va_cache = build_chunk_cache_MTL(text_te, y_u_te, y_s_te, tokenizer, "val", MAX_LEN, STRIDE)

# Dataset & DataLoader
class MTLChunkDataset(torch.utils.data.Dataset):
    def __init__(self, tensors):
        self.t = tensors

    def __len__(self):
        return self.t["y_u"].size(0)

    def __getitem__(self, i):
        return {
            "input_ids": self.t["input_ids"][i].long(),
            "attention_mask": self.t["attention_mask"][i].bool(),
            "y_u": self.t["y_u"][i],
            "y_s": self.t["y_s"][i],
        }

from torch.utils.data import DataLoader

def make_loader(ds, batch_size, shuffle):
    try:
        return DataLoader(
            ds, batch_size=batch_size, shuffle=shuffle,
            num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY,
            prefetch_factor=PREFETCH, persistent_workers=PERSISTENT
        )
    except Exception as e:
        print(f"⚠️ DataLoader fallback (workers=0): {e}")
        return DataLoader(ds, batch_size=batch_size, shuffle=shuffle,
                         num_workers=0, pin_memory=False)

print("\n🚚 Creating DataLoaders...")
train_loader = make_loader(MTLChunkDataset(tr_cache), BATCH_SIZE, True)
val_loader = make_loader(MTLChunkDataset(va_cache), BATCH_SIZE * 2, False)  # MODIFIED: 필요 시 BATCH_SIZE * 1.5로 조정 가능
print(f"  Train batches: {len(train_loader):,}")
print(f"  Val batches: {len(val_loader):,}")

# 🔧 Optimizer / Loss / Scheduler
from torch.optim import AdamW
from transformers import get_cosine_schedule_with_warmup
from torch.optim.lr_scheduler import ReduceLROnPlateau  # MODIFIED: 추가 (plateau 시 LR 감소)

def make_class_weights(y_np, gamma=0.5, device='cpu'):
    """클래스 빈도 기반 가중치 (gamma로 조정)"""
    cnt = torch.bincount(torch.tensor(y_np, dtype=torch.long))
    w = (cnt.sum() / (len(cnt) * cnt.float().clamp(min=1))).pow(gamma)
    return w.to(device)

print("\n⚖️ Computing class weights...")
wu = make_class_weights(y_u_tr.values, GAMMA_URGENCY, device=device)
ws = make_class_weights(y_s_tr.values, GAMMA_SENT, device=device)
print(f"  Urgency weights: {wu.tolist()}")
print(f"  Sentiment weights: {ws.tolist()}")

loss_u = nn.CrossEntropyLoss(weight=wu, label_smoothing=LABEL_SMOOTHING).to(device)
loss_s = nn.CrossEntropyLoss(weight=ws, label_smoothing=LABEL_SMOOTHING).to(device)

# Optimizer (weight decay 제외 파라미터)
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters()
                   if not any(nd in n for nd in no_decay)],
        "weight_decay": WEIGHT_DECAY
    },
    {
        "params": [p for n, p in model.named_parameters()
                   if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=LR, eps=1e-8)

# Scheduler
total_steps = max(1, EPOCHS * len(train_loader) // GRAD_ACCUM)
warmup_steps = int(total_steps * WARMUP_RATIO)
scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
reduce_lr = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)
print(f"\n📅 Scheduler: {total_steps:,} total steps, {warmup_steps:,} warmup steps")

# GradScaler
from torch.cuda.amp import GradScaler
scaler = GradScaler(enabled=torch.cuda.is_available())

# 📏 Evaluation Functions
from sklearn.metrics import f1_score, classification_report  # MODIFIED: classification_report 추가 (PR/Recall 출력)
from torch.amp import autocast

@torch.no_grad()
def evaluate_call_level(model, loader, cache_tensors):
    """Call-level 평가 (청크 로짓 평균)"""
    model.eval()
    logits_u_list, logits_s_list = [], []
    labels_u_list, labels_s_list = [], []

    for batch in tqdm(loader, desc="Eval", leave=False):
        ids = batch["input_ids"].to(device, non_blocking=True)
        mask = batch["attention_mask"].to(device, non_blocking=True)
        yu = batch["y_u"].to(device, non_blocking=True)
        ys = batch["y_s"].to(device, non_blocking=True)

        with autocast('cuda', dtype=amp_dtype, enabled=torch.cuda.is_available()):
            lu, ls = model(ids, mask)

        # MODIFIED: FP32 캐스팅 강화
        logits_u_list.append(lu.detach().float().cpu().numpy())
        logits_s_list.append(ls.detach().float().cpu().numpy())
        labels_u_list.append(yu.detach().cpu().numpy())
        labels_s_list.append(ys.detach().cpu().numpy())

    # 연결
    all_logits_u = np.concatenate(logits_u_list, 0)
    all_logits_s = np.concatenate(logits_s_list, 0)
    all_labels_u = np.concatenate(labels_u_list, 0)
    all_labels_s = np.concatenate(labels_s_list, 0)

    idx = cache_tensors["sample_idx"].cpu().numpy()

    # Call-level 집계
    results = {}
    for task_name, logits, ytrue in [
        ("urgency", all_logits_u, all_labels_u),
        ("sentiment", all_logits_s, all_labels_s)
    ]:
        preds, gts = [], []
        for sid in np.unique(idx):
            mask = (idx == sid)
            pooled_logits = logits[mask].mean(0)
            preds.append(pooled_logits.argmax())
            gts.append(ytrue[mask][0])

        preds = np.array(preds)
        gts = np.array(gts)

        results[task_name] = {
            "micro": f1_score(gts, preds, average="micro"),
            "macro": f1_score(gts, preds, average="macro"),
            "weighted": f1_score(gts, preds, average="weighted"),
            "per_class": f1_score(gts, preds, average=None).tolist(),
            "predictions": preds,
            "ground_truth": gts,
            "report": classification_report(gts, preds, output_dict=True)  # MODIFIED: PR/Recall 추가
        }

    return results

# 🏃 Training Loop
print("\n🏃 Starting training...")
best_combo = -1.0
best_epoch = 0
patience = 3  # MODIFIED: 5→3으로 줄여 오버피팅 방지
patience_counter = 0
history = []

for epoch in range(1, EPOCHS + 1):
    model.train()
    running_loss = 0.0
    optimizer.zero_grad(set_to_none=True)

    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=True)  # MODIFIED: leave=True로 출력 개선
    for step, batch in enumerate(pbar):
        ids = batch["input_ids"].to(device, non_blocking=True)
        mask = batch["attention_mask"].to(device, non_blocking=True)
        tu = batch["y_u"].to(device, non_blocking=True)
        ts = batch["y_s"].to(device, non_blocking=True)

        with autocast('cuda', dtype=amp_dtype, enabled=torch.cuda.is_available()):
            lu, ls = model(ids, mask)
            loss = (ALPHA * loss_u(lu, tu) + BETA * loss_s(ls, ts)) / GRAD_ACCUM

        scaler.scale(loss).backward()

        if (step + 1) % GRAD_ACCUM == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            optimizer.zero_grad(set_to_none=True)

        running_loss += loss.item() * GRAD_ACCUM
        pbar.set_postfix({
            "loss": f"{loss.item() * GRAD_ACCUM:.4f}",
            "lr": f"{scheduler.get_last_lr()[0]:.2e}"
        })

    avg_loss = running_loss / len(train_loader)

    # Call-level 평가
    val_results = evaluate_call_level(model, val_loader, va_cache)

    urgW = val_results['urgency']['weighted']
    sentW = val_results['sentiment']['weighted']
    combo = 0.5 * urgW + 0.5 * sentW

    # MODIFIED: 클래스별 상세 보고서 출력 (PR/Recall 포함)
    print(f"\n{'='*70}")
    print(f"Epoch {epoch}/{EPOCHS} | Loss: {avg_loss:.4f}")
    print(f"{'='*70}")
    print(f"Urgency  → Weighted F1: {urgW:.4f} | Per-class F1: {val_results['urgency']['per_class']}")
    print(f"  Report: {val_results['urgency']['report']}")
    print(f"Sentiment → Weighted F1: {sentW:.4f} | Per-class F1: {val_results['sentiment']['per_class']}")
    print(f"  Report: {val_results['sentiment']['report']}")
    print(f"Combo (0.5U + 0.5S): {combo:.4f}")

    # 히스토리 저장
    history.append({
        "epoch": epoch,
        "loss": avg_loss,
        "urg_weighted": urgW,
        "sent_weighted": sentW,
        "combo": combo,
        "urg_per_class": val_results['urgency']['per_class'],
        "sent_per_class": val_results['sentiment']['per_class']
    })

    # Best model 저장
    if combo > best_combo:
        best_combo = combo
        best_epoch = epoch
        patience_counter = 0

        torch.save({
            "state_dict": model.state_dict(),
            "history": history,
            "best_combo": best_combo,
            "epoch": epoch,
            "config": {
                "alpha": ALPHA, "beta": BETA,
                "max_len": MAX_LEN, "stride": STRIDE,
                "gamma_u": GAMMA_URGENCY, "gamma_s": GAMMA_SENT,
                "dropout": DROPOUT_RATE,
                "lr": LR, "batch_size": BATCH_SIZE
            }
        }, SAVE_DIR / "best_mtl.pt")

        print(f"✅ Best model saved! (combo={best_combo:.4f})")
    else:
        patience_counter += 1
        print(f"⏳ No improvement ({patience_counter}/{patience})")

        if patience_counter >= patience:
            print(f"\n⛔ Early stopping at epoch {epoch}")
            break

    # MODIFIED: Reduce LR on plateau (combo 기반)
    reduce_lr.step(combo)

# ✅ Best model 복원
print(f"\n{'='*70}")
print("🏆 Training completed!")
print(f"{'='*70}")
ckpt_path = SAVE_DIR / "best_mtl.pt"
if ckpt_path.exists():
    state = torch.load(ckpt_path, map_location=device, weights_only=False)
    model.load_state_dict(state["state_dict"])
    model.to(device)
    print(f"✅ Loaded best model from epoch {state['epoch']}")
    print(f"   Best combo: {state['best_combo']:.4f}")

    # Final evaluation
    print("\n📊 Final Evaluation on Test Set:")
    final_results = evaluate_call_level(model, val_loader, va_cache)

    print(f"\nUrgency:")
    print(f"  Weighted F1: {final_results['urgency']['weighted']:.4f}")
    print(f"  Per-class F1: {final_results['urgency']['per_class']}")
    print(f"  Report: {final_results['urgency']['report']}")

    print(f"\nSentiment:")
    print(f"  Weighted F1: {final_results['sentiment']['weighted']:.4f}")
    print(f"  Per-class F1: {final_results['sentiment']['per_class']}")
    print(f"  Report: {final_results['sentiment']['report']}")

    # ✅ 혼동 행렬 출력
    from sklearn.metrics import confusion_matrix

    print("\n📈 Confusion Matrix (Urgency):")
    cm_u = confusion_matrix(final_results['urgency']['ground_truth'],
                            final_results['urgency']['predictions'])
    print(cm_u)

    print("\n📈 Confusion Matrix (Sentiment):")
    cm_s = confusion_matrix(final_results['sentiment']['ground_truth'],
                            final_results['sentiment']['predictions'])
    print(cm_s)

# 🎯 Inference Function
@torch.no_grad()
def predict_text(texts, aggregate="mean"):
    """긴 텍스트 예측 (청크 평균)"""
    if isinstance(texts, str):
        texts = [texts]

    model.eval()
    results = []

    for text in texts:
        enc = tokenizer(
            text,
            max_length=MAX_LEN,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            return_overflowing_tokens=True,
            stride=STRIDE
        ).to(device)

        with autocast('cuda', dtype=amp_dtype, enabled=torch.cuda.is_available()):
            lu, ls = model(
                enc["input_ids"],
                enc["attention_mask"]
            )
            prob_u = lu.softmax(-1).float()  # MODIFIED: FP32 캐스팅
            prob_s = ls.softmax(-1).float()  # MODIFIED: FP32 캐스팅

        # 청크 평균
        if aggregate == "mean":
            avg_u = prob_u.mean(0)
            avg_s = prob_s.mean(0)
        else:  # max
            avg_u = prob_u.max(0).values
            avg_s = prob_s.max(0).values

        pred_u = int(avg_u.argmax().item())
        pred_s = int(avg_s.argmax().item())
        conf_u = float(avg_u.max().item())
        conf_s = float(avg_s.max().item())

        # MODIFIED: Confidence threshold 예시 (0.5 미만 시 경고)
        if conf_u < 0.5 or conf_s < 0.5:
            print("⚠️ Low confidence prediction")

        results.append({
            "urgency": pred_u,
            "sentiment": pred_s,
            "urgency_conf": conf_u,
            "sentiment_conf": conf_s
        })

    return results[0] if len(results) == 1 else results

# 테스트
print("\n🧪 Test Inference:")
test_text = "[TURN=12] [TURN_BIN=M] [IRESP=UNK] [IRESP_BIN=UNK] [CALLER] 안녕하세요 급한 일이 있어서요 [AGENT] 네 말씀하세요"
result = predict_text(test_text)
print(f"Input: {test_text[:100]}...")
print(f"Prediction: {result}")

print("\n✅ All done!")

PyTorch: 2.8.0+cu126 | CUDA: 12.6
GPU: NVIDIA A100-SXM4-80GB
Python: 3.12.12
Device: cuda

📋 Config:
  AMP dtype: torch.bfloat16
  MAX_LEN/STRIDE: 384/192
  BATCH: 160 | GRAD_ACCUM: 1
  Workers: 4 | Prefetch: 4
  Task weights: α=0.4, β=0.6
  Class gamma: Urgency=0.3, Sentiment=0.8

📂 Loading data...
  → X_train.parquet
  → X_test.parquet
  → y_train_urgency.parquet
  → y_test_urgency.parquet
  → y_train_sentiment.parquet
  → y_test_sentiment.parquet
✅ Data loaded: Train=101,742, Test=25,436

📊 Class Distribution:
Urgency (Train): {0: 0.35418018124275125, 1: 0.33681272237620646, 2: 0.30900709638104223}
Sentiment (Train): {0: 0.02251774095260561, 1: 0.37563641367380235, 2: 0.535884885298107, 3: 0.06596096007548505}

🔤 Loading tokenizer: beomi/KcELECTRA-base-v2022
🏗️ Building model...
  Total params: 127,191,559
  Trainable params: 127,191,559

🔄 Preparing chunks...
⚡ Loading cached chunks: train_ml384_st192.pt
  Chunks: 157,272
⚡ Loading cached chunks: val_ml384_st192.pt
  Chunks: 39,466

Epoch 1/15:   0%|          | 0/983 [00:00<?, ?it/s]

Eval:   0%|          | 0/124 [00:00<?, ?it/s]


Epoch 1/15 | Loss: 1.3430
Urgency  → Weighted F1: 0.5310 | Per-class F1: [0.6027095292766934, 0.3612992398064962, 0.6337594858822725]
  Report: {'0': {'precision': 0.5140216199279336, 'recall': 0.7283827283827283, 'f1-score': 0.6027095292766934, 'support': 9009.0}, '1': {'precision': 0.44282568185668303, 'recall': 0.3051243142290183, 'f1-score': 0.3612992398064962, 'support': 8567.0}, '2': {'precision': 0.6849416284912073, 'recall': 0.5896946564885496, 'f1-score': 0.6337594858822725, 'support': 7860.0}, 'accuracy': 0.5429705928605126, 'macro avg': {'precision': 0.547262976758608, 'recall': 0.5410672330334321, 'f1-score': 0.5325894183218206, 'support': 25436.0}, 'weighted avg': {'precision': 0.5428585308357385, 'recall': 0.5429705928605126, 'f1-score': 0.530995844303768, 'support': 25436.0}}
Sentiment → Weighted F1: 0.6544 | Per-class F1: [0.1900214848412509, 0.642612617508015, 0.7184785740748364, 0.3586656634060697]
  Report: {'0': {'precision': 0.11006637168141593, 'recall': 0.694589

Epoch 2/15:   0%|          | 0/983 [00:00<?, ?it/s]

...

Epoch 6/15:   0%|          | 0/983 [00:00<?, ?it/s]

Eval:   0%|          | 0/124 [00:00<?, ?it/s]


Epoch 6/15 | Loss: 0.9920
Urgency  → Weighted F1: 0.5754 | Per-class F1: [0.592629871945423, 0.4959173471088152, 0.6421993860936874]
  Report: {'0': {'precision': 0.5971377056569754, 'recall': 0.5881895881895882, 'f1-score': 0.592629871945423, 'support': 9009.0}, '1': {'precision': 0.47308181432810514, 'recall': 0.5210692190965333, 'f1-score': 0.4959173471088152, 'support': 8567.0}, '2': {'precision': 0.6752736458040977, 'recall': 0.6122137404580152, 'f1-score': 0.6421993860936874, 'support': 7860.0}, 'accuracy': 0.5730067620695077, 'macro avg': {'precision': 0.5818310552630593, 'recall': 0.5738241825813789, 'f1-score': 0.5769155350493086, 'support': 25436.0}, 'weighted avg': {'precision': 0.5794997778594424, 'recall': 0.5730067620695077, 'f1-score': 0.5753740212192923, 'support': 25436.0}}
Sentiment → Weighted F1: 0.6603 | Per-class F1: [0.25684420109507217, 0.6278551532033426, 0.7428682170542635, 0.3117543516526501]
  Report: {'0': {'precision': 0.1796657381615599, 'recall': 0.45026

Eval:   0%|          | 0/124 [00:00<?, ?it/s]


Urgency:
  Weighted F1: 0.5817
  Per-class F1: [0.5863317757009345, 0.4967496077112755, 0.6688458637163691]
  Report: {'0': {'precision': 0.6187892984835409, 'recall': 0.5571095571095571, 'f1-score': 0.5863317757009345, 'support': 9009.0}, '1': {'precision': 0.47774064891667567, 'recall': 0.5173339558772032, 'f1-score': 0.4967496077112755, 'support': 8567.0}, '2': {'precision': 0.6610337972166997, 'recall': 0.6768447837150128, 'f1-score': 0.6688458637163691, 'support': 7860.0}, 'accuracy': 0.5807123761597736, 'macro avg': {'precision': 0.5858545815389721, 'recall': 0.5837627655672577, 'f1-score': 0.5839757490428598, 'support': 25436.0}, 'weighted avg': {'precision': 0.584337261182208, 'recall': 0.5807123761597736, 'f1-score': 0.5816577034660668, 'support': 25436.0}}

Sentiment:
  Weighted F1: 0.6728
  Per-class F1: [0.29120409906063194, 0.6435676763160836, 0.7492552726695, 0.34804909324052025]
  Report: {'0': {'precision': 0.19276427360090448, 'recall': 0.5951134380453752, 'f1-score':